In [1]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the path to your dataset
data_dir = 'D:\\project\\Data Analysis\\car color detection\\data'

# Function to load and preprocess images from a given directory
def load_images_from_directory(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for image_file in os.listdir(label_dir):
                if image_file.endswith('.jpg'):
                    image_path = os.path.join(label_dir, image_file)
                    image = load_img(image_path, target_size=(128, 128))  # Resize images to 128x128
                    image = img_to_array(image)
                    images.append(image)
                    labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess the dataset
train_images, train_labels = load_images_from_directory(os.path.join(data_dir, 'train'))
val_images, val_labels = load_images_from_directory(os.path.join(data_dir, 'val'))
test_images, test_labels = load_images_from_directory(os.path.join(data_dir, 'test'))

# Normalize the images
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)

# Optionally, split the training data into training and validation sets if you don't have a separate validation set
# X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

print(f'Training data: {train_images.shape}, {train_labels.shape}')
print(f'Validation data: {val_images.shape}, {val_labels.shape}')
print(f'Test data: {test_images.shape}, {test_labels.shape}')




Training data: (7267, 128, 128, 3), (7267,)
Validation data: (1550, 128, 128, 3), (1550,)
Test data: (1556, 128, 128, 3), (1556,)


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define the model
model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(train_labels)), activation='softmax')  # Use train_labels instead of labels
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           1,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,306,575 (12.61 MB)

 Trainable params: 3,306,575 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# # Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

datagen.fit(train_images)

# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32), 
                    epochs=35, 
                    validation_data=(val_images, val_labels))

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {accuracy}')


Epoch 1/35


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


228/228 ━━━━━━━━━━━━━━━━━━━━ 71s 298ms/step - accuracy: 0.3412 - loss: 2.0219 - val_accuracy: 0.6394 - val_loss: 0.9907
Epoch 2/35
228/228 ━━━━━━━━━━━━━━━━━━━━ 67s 292ms/step - accuracy: 0.5893 - loss: 1.2001 - val_accuracy: 0.6774 - val_loss: 0.8664
Epoch 3/35
228/228 ━━━━━━━━━━━━━━━━━━━━ 67s 292ms/step - accuracy: 0.6173 - loss: 1.1004 - val_accuracy: 0.7432 - val_loss: 0.7289
Epoch 4/35
228/228 ━━━━━━━━━━━━━━━━━━━━ 68s 294ms/step - accuracy: 0.6733 - loss: 0.9723 - val_accuracy: 0.7665 - val_loss: 0.6779
Epoch 5/35
228/228 ━━━━━━━━━━━━━━━━━━━━ 69s 297ms/step - accuracy: 0.6803 - loss: 0.9180 - val_accuracy: 0.7606 - val_loss: 0.6390
Epoch 6/35
228/228 ━━━━━━━━━━━━━━━━━━━━ 71s 308ms/step - accuracy: 0.7035 - loss: 0.8620 - val_accuracy: 0.7432 - val_loss: 0.7588
Epoch 7/35
228/228 ━━━━━━━━━━━━━━━━━━━━ 68s 293ms/step - accuracy: 0.6980 - loss: 0.8560 - val_accuracy: 0.7213 - val_loss: 0.7104
Epoch 8/35
228/228 ━━━━━━━━━━━━━━━━━━━━ 69s 300ms/step - accuracy: 0.7125 - loss: 0.8426 - val

In [5]:
# Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f'Test accuracy: {accuracy}')

model.save(r'D:\project\Data Analysis\car color detection\models\COLOR_detection_model.keras')

# Save the model weights
model.save_weights(r'D:\project\Data Analysis\car color detection\models\AGE_detection_weights.weights.h5')

In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Function to predict dress color
def predict_dress_color(image_path):
    image = load_img(image_path, target_size=(128, 128))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

# Example usage
image_path = r"D:\project\Data Analysis\car color detection\data\val\grey\6ae519605b.jpg"
predicted_color = predict_dress_color(image_path)
print(f'Predicted Dress Color: {predicted_color}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted Dress Color: silver


In [8]:
# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)

# Save the classes (car colors)
np.save(r'D:\project\Data Analysis\car color detection\models\car_color_classes.npy', label_encoder.classes_)
